In [ ]:
!pip install -r requirements.txt

In [ ]:
#  # ДАННЫЙ КОД НУЖЕН ДЛЯ ПОДГОТОВКИ ОКРУЖЕНИЯ В ПЕРВЫЙ РАЗ
#  
#  import sys
#  import torch
#  from manage_dataset_structure import decompose_json
#  sys.path.append('src')
#  from wiki_bench import WikiBench
#  from sentence_transformers import SentenceTransformer
#  
#  decompose_json() # json -> файлы
#  
#  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#  encoder = SentenceTransformer('sergeyzh/BERTA').to(device)
#  
#  bench = WikiBench(
#      key='',
#      url='',
#      model_name='',
#      device=device,
#      encoder=encoder
#  )
#  
#  bench.prepare_env()

In [1]:
import sys
import logging
import torch
sys.path.append('src')
from wiki_bench import WikiBench
from sentence_transformers import SentenceTransformer

with open('Access_key.txt', 'r', encoding='utf-8') as file:
    url, key = file.read().split()

logging.getLogger('sentence_transformers.SentenceTransformer').setLevel(logging.ERROR)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
encoder = SentenceTransformer('sergeyzh/BERTA').to(device)

bench = WikiBench(
    key=key,
    url=url,
    model_name='ruadapt-qwen3-4b', # example model name
    device=device, # 'cuda'
    encoder=encoder, # 'sergeyzh/BERTA' - default
    number_of_articles=3 # число статей для подсчета
)

# подготовка окружения (корпус уже должен быть готов!)
bench.load_enviroment()

# Ранжирование источников
score_query = await bench.rank_query()

# Генерация плана
score_outline = await bench.rank_outline(
    neighbor_count=0,
    description_mode=0,
    clusterization_with_hint=True             
)

# Генерация секций
score_sections = await bench.rank_sections()

resource module not available on Windows


18:31:23 | INFO | wikibench | WikiBench initialized: model=ruadapt-qwen3-4b, articles=3
18:31:23 | INFO | wikibench | Loading enviroment...
18:31:23 | INFO | wikibench | Enviroment loaded!
18:31:23 | INFO | wikibench | Stage: rank_query started
rank_query: 100%|███████████████████████████████████████████████████████████████████| 3/3 [01:17<00:00, 25.95s/article]
18:32:41 | INFO | wikibench | Final result for stage rank_query: (0.9810372896996004, 0.8924301290215108)
18:32:41 | INFO | wikibench | Stage: rank_outline started: neighbor_count=0, description_mode=0, clusterization_with_hint=True


                  NDCG  R-Precision
Atomic Heart  0.998011     0.942029
ChatGPT       0.953926     0.838710
F-35          0.991175     0.896552


rank_outline: 100%|█████████████████████████████████████████████████████████████████| 3/3 [00:26<00:00,  8.78s/article]
18:33:07 | INFO | wikibench | Final result for stage rank_outline: P=0.5226 [0.4759; 0.5563] | R=0.6499 [0.6100; 0.7220] | F=0.5782 [0.5376; 0.6150]
18:33:07 | INFO | wikibench | Stage: rank_sections started


              Precision    Recall  F1-Score
Atomic Heart   0.475889  0.617766  0.537625
ChatGPT        0.535635  0.722001  0.615009
F-35           0.556281  0.609997  0.581902


rank_sections: 100%|████████████████████████████████████████████████████████████████| 3/3 [00:58<00:00, 19.39s/article]
18:34:06 | INFO | wikibench | Final result for stage rank_sections: P=0.5244 [0.4694; 0.5739] | R=0.6156 [0.5394; 0.6982] | F=0.5532 [0.5030; 0.6016] | Rouge=0.5532 [0.5030; 0.6016] | BLEU=0.5532 [0.5030; 0.6016]


 article_id        topic  precision   recall       f1  rouge_l     bleu  num_sections
          1 Atomic Heart   0.459484 0.544824 0.490926 0.150626 0.044010             5
          2      ChatGPT   0.597180 0.734869 0.648931 0.294507 0.120031             4
          3         F-35   0.527178 0.601580 0.543679 0.185245 0.036245             7
